In [613]:
from yahoofinancials import YahooFinancials
import numpy as np

In [760]:
class cookFinancials(YahooFinancials):
    ticker = ''
    bshData = []
    def __init__(self, ticker):
        if isinstance(ticker, str):
            self.ticker = ticker.upper()
        else:
            self.ticker = [t.upper() for t in ticker]
        self._cache = {}
        
    def get_balanceSheetHistory(self):
        self.bshData = self.get_financial_stmts('annual', 'balance')['balanceSheetHistory']
        return self.bshData
    
    def get_BV(self, numofYears=20):
        bv = []
        if not(self.bshData):
            self.get_balanceSheetHistory()
        for i in range(min(np.size(self.bshData[self.ticker]), numofYears)):
            date_key = list(self.bshData[self.ticker][i].keys())[0]
            if not(self.bshData[self.ticker][i][date_key]):    
                break
            bv.append(self.bshData[self.ticker][i][date_key]['totalStockholderEquity'])
        return bv   
    #use mean of each year    
    def get_BV_GR_median(self, bv):
        gr = []
        for v in range(np.size(bv)-1):
            gr.append((bv[v]-bv[v+1])/abs(bv[v+1]))
        #print(gr)
        return np.size(bv)-1, np.median(gr)
    
    def get_BV_GR_max(self, bv):
        gr = []
        for v in range(np.size(bv)-1):
            gr.append((bv[v]-bv[v+1])/abs(bv[v+1]))
        #print(gr)
        return np.size(bv)-1, np.max(gr)
    
    def get_BV_GR_mean(self, bv):
        gr = []
        for v in range(np.size(bv)-1):
            gr.append((bv[v]-bv[v+1])/abs(bv[v+1]))
        #print(gr)
        return np.size(bv)-1, np.mean(gr)
    
    def get_suggest_price(self, cEPS, growth, years, rRate, PE, safty):
        if not(cEPS) or not(growth) or not(PE):
            return 'NA'
        fEPS = cEPS*(1+growth)**years
        fPrice = fEPS*PE;
        stickerPrice = fPrice/(1+rRate)**years
        return stickerPrice, stickerPrice*safty
    
    def get_PE(self):
        #print(self._stock_summary_data('trailingPE'))
        #print(self._stock_summary_data('forwardPE'))
        if not(self._stock_summary_data('trailingPE')):
            return self._stock_summary_data('forwardPE')
        if not(self._stock_summary_data('forwardPE')):
            return self._stock_summary_data('trailingPE')
        return (self._stock_summary_data('trailingPE')+self._stock_summary_data('forwardPE'))/2
    
    def get_decision(self,suggestPrice, stockprice):
        print('suggested price:', suggestPrice)
        print('stock price:', stockprice)
        if isinstance(suggestPrice, str):
            return 'no recommendation due to error'
        elif suggestPrice>stockprice:
            return 'strong buy' 
        else:
            return 'do not buy'
            
            
        

In [761]:
class batch_process:
    def __init__(self, tickers):
        self.tickers = tickers
    def batch_run(self):
        for i in range(np.size(self.tickers)):
            print(self.tickers[i])
            x = cookFinancials(self.tickers[i])
            bv = x.get_BV(20)
            bv.insert(0, x.get_book_value())
            #print(bv)
            bvgr = x.get_BV_GR_median(bv)
            print(bvgr)
            growth = bvgr[1]
            cEPS = x.get_earnings_per_share()
            years = 3;
            rRate = 0.2;
            safty = 0.5
            PE = x.get_PE()
            price=x.get_suggest_price(cEPS, growth, years, rRate, PE, safty)
            #print(price)
            print(x.get_decision(price[1],x.get_current_price()))
            print('=====================================')

In [743]:
x = cookFinancials('SGEN')

In [744]:
bv = x.get_BV(20)
bv

[1876287000, 1273943000, 677569000, 634087000]

In [745]:
bv.insert(0, x.get_book_value())
print(bv)

[2258030000, 1876287000, 1273943000, 677569000, 634087000]


In [746]:
bvgr = x.get_BV_GR_median(bv)
print(bvgr)

(4, 0.3381376283195572)


In [747]:
growth = bvgr[1]
cEPS = x.get_earnings_per_share()
years = 3;
rRate = 0.2;
safty = 0.5
PE = x.get_PE()

In [748]:
print(x.get_earnings_per_share())


2.6480002393131707


In [749]:
x.get_pe_ratio()
x.get_PE()

66.858

In [750]:
print(cEPS)
print(growth)
print(years)
print(rRate)
print(PE)
print(safty)

2.6480002393131707
0.3381376283195572
3
0.2
66.858
0.5


In [751]:
price=x.get_suggest_price(cEPS, growth, years, rRate, PE, safty)
print(price)

(245.48785567341707, 122.74392783670854)


In [752]:
x.get_decision(price[1],x.get_current_price())

suggested price: 122.74392783670854
stock price: 177.04


'do not buy'

In [628]:
from get_all_tickers import get_tickers as gt
from get_all_tickers.get_tickers import Region

In [775]:
tickers = gt.get_tickers()



In [774]:
tickers

['DDD',
 'MMM',
 'WBAI',
 'EGHT',
 'AHC',
 'AOS',
 'ATEN',
 'AIR',
 'ABB',
 'ABT',
 'ABBV',
 'ANF',
 'AGD',
 'AWP',
 'ACP',
 'JEQ',
 'ASGI',
 'AOD',
 'ABM',
 'AKR',
 'ACEL',
 'ACN',
 'ACCO',
 'ATV',
 'AYI',
 'GOLF',
 'ADX',
 'PEO',
 'ADCT',
 'AGRO',
 'ADNT',
 'ADT',
 'ATGE',
 'AAP',
 'WMS',
 'ASIX',
 'AVK',
 'IMPX',
 'ACM',
 'AEFC',
 'AEB',
 'AEG',
 'AENZ',
 'AER',
 'AJRD',
 'AMG',
 'MGR',
 'MGRB',
 'AFL',
 'MITT',
 'AGCO',
 'A',
 'AEM',
 'ADC',
 'AL',
 'APD',
 'ALG',
 'AGI',
 'ALK',
 'AIN',
 'ALB',
 'ACI',
 'AA',
 'ALC',
 'ALEX',
 'ALX',
 'ARE',
 'AQN',
 'AQNA',
 'AQNB',
 'BABA',
 'Y',
 'ATI',
 'ALLE',
 'ALE',
 'ADS',
 'AFB',
 'AWF',
 'AB',
 'AIO',
 'CBH',
 'NCV',
 'NCZ',
 'ACV',
 'NFJ',
 'NIE',
 'ALSN',
 'ALL',
 'ALLY',
 'PINE',
 'ALTG',
 'AYX',
 'ATUS',
 'MO',
 'ACH',
 'ALUS',
 'AMBC',
 'ABEV',
 'AMC',
 'AMCR',
 'AEE',
 'AMRC',
 'AMOV',
 'AMX',
 'AAT',
 'AXL',
 'ACC',
 'AEO',
 'AEL',
 'AXP',
 'AFG',
 'AFGB',
 'AFGC',
 'AFGD',
 'AFGE',
 'AMH',
 'AIG',
 'ARL',
 'ARA',
 'AWR',
 'AMT',


In [782]:
filtered_by_sector = gt.get_tickers_filtered(mktcap_min=10e3, sectors=gt.SectorConstants.TECH)

In [784]:
print(filtered_by_sector)

['ANET', 'ASX', 'ATHM', 'AVLR', 'BILL', 'BKI', 'CDAY', 'CCC', 'NET', 'DELL', 'DOV', 'DT', 'ETN', 'ESTC', 'EPAM', 'FDS', 'GDDY', 'GWRE', 'HPE', 'HPQ', 'HUBS', 'INFO', 'ITW', 'INFY', 'IR', 'IBM', 'LDOS', 'LN', 'MSI', 'NOK', 'OMC', 'ORCL', 'PAGS', 'PLTR', 'PANW', 'PAYC', 'PINS', 'RNG', 'CRM', 'SAP', 'NOW', 'SHOP', 'WORK', 'SNAP', 'SNOW', 'SQ', 'STM', 'TSM', 'TEL', 'TWLO', 'TWTR', 'TYL', 'UI', 'UMC', 'URI', 'U', 'VEEV', 'VMW', 'WIT', 'WPP', 'ZEN', 'ATVI', 'ADBE', 'AMD', 'API', 'GOOG', 'GOOGL', 'ADI', 'ANSS', 'AAPL', 'AMAT', 'ASML', 'TEAM', 'ADSK', 'ADP', 'BIDU', 'BILI', 'AVGO', 'CDNS', 'CERN', 'CHKP', 'CSCO', 'CTXS', 'CTSH', 'COUP', 'CRWD', 'DADA', 'DDOG', 'DOCU', 'EA', 'ENPH', 'ERIC', 'FFIV', 'FB', 'FISV', 'FIVN', 'FTNT', 'GDS', 'GDRX', 'INFO', 'INTC', 'INTU', 'IPGP', 'JKHY', 'LRCX', 'LOGI', 'MRVL', 'MTCH', 'MXIM', 'MCHP', 'MU', 'MSFT', 'MDB', 'MPWR', 'NTAP', 'NICE', 'NLOK', 'NUAN', 'NVDA', 'NXPI', 'OKTA', 'ON', 'OTEX', 'PCTY', 'PEGA', 'PTC', 'QRVO', 'QCOM', 'STX', 'SWKS', 'SEDG', 'SPLK',

In [753]:
filtered_by_sector.index('SGEN')

90

In [785]:
y = batch_process(filtered_by_sector[:])

In [786]:
y.batch_run()


ANET
(4, 0.3201147041600926)
suggested price: 172.9170330706802
stock price: 271.58
do not buy
ASX
(4, 0.04332585046766976)
suggested price: 1.666575356176278
stock price: 5.45
do not buy
ATHM
(4, 0.2819696245360388)
suggested price: 46.390656508311025
stock price: 92.78
do not buy
AVLR
(4, 1.289310288914812)
suggested price: A
stock price: 168.23
no recommendation due to error
BILL
(3, 0.011325151009048583)
suggested price: A
stock price: 117.7
no recommendation due to error
BKI
(4, 0.21143575204576565)
suggested price: 37.9083479140525
stock price: 89.46
do not buy
CDAY
(4, 0.19818018939899384)
suggested price: 26.523090471780932
stock price: 93.9
do not buy
CCC
(3, 0.2948819111237599)
suggested price: A
stock price: 27
no recommendation due to error
NET
(3, 0.13862650655527206)
suggested price: A
stock price: 71.9
no recommendation due to error
DELL
(4, 0.3059467244440582)
suggested price: 32.44364548402994
stock price: 70.52
do not buy
DOV
(4, 0.08291845932291372)
suggested price: 

(4, 0.1235947138418104)
suggested price: 29.31607754430174
stock price: 79.15
do not buy
ADBE
(4, 0.11854591630949468)
suggested price: 166.70361057618112
stock price: 481.26
do not buy
AMD
(4, 0.7784266907589056)
suggested price: 107.04663986308552
stock price: 93.74
strong buy
API
(2, 2.70804582382348)
suggested price: A
stock price: 37.27
no recommendation due to error
GOOG
(4, 0.11545965745365708)
suggested price: 676.9240693284193
stock price: 1827.95
do not buy
GOOGL
(4, 0.11545965745365708)
suggested price: 675.8709754862167
stock price: 1824.97
do not buy
ADI
(4, 0.03189941496378224)
suggested price: 33.946785846537985
stock price: 140.66
do not buy
ANSS
(4, 0.10960594571688134)
suggested price: A
stock price: None
no recommendation due to error
AAPL
(4, -0.1780769352621166)
suggested price: 17.376969323711943
stock price: 123.08
do not buy
AMAT
(4, 0.1)
suggested price: 28.457609273094334
stock price: 85.29
do not buy
ASML
(4, 0.08097103403559353)
suggested price: 132.68286893

suggested price: 9.361543476408318
stock price: 44.78
do not buy
PCTY
(4, 0.358797240408949)
suggested price: 108.57780085373676
stock price: 194.77
do not buy
PEGA
(4, -0.03516505230210155)
suggested price: A
stock price: 127.8
no recommendation due to error
PTC
(4, 0.09827387399979035)
suggested price: 27.061301464241147
stock price: 107.26
do not buy
QRVO
(4, -0.0200569949875775)
suggested price: 30.378555352777738
stock price: 159.55
do not buy
QCOM
(4, 0.11896516602159299)
suggested price: 47.90483615055828
stock price: 149.98
do not buy
STX
(4, 0.11985039393337676)
suggested price: 21.555431471564443
stock price: 61.2
do not buy
SWKS
(4, 0.006936900739020365)
suggested price: 33.02866443858077
stock price: 143.45
do not buy
SEDG
(4, 0.39567799366577133)
suggested price: 186.39981617120412
stock price: 273.5
do not buy
SPLK
(4, 0.3294922406385141)
suggested price: A
stock price: 205.91
no recommendation due to error
SSNC
(4, 0.15323667799657892)
suggested price: 23.094938809431067